# Analysis of Citibike Data in NYC

## Andrew Li, Arun Srinivas, and Matt Wong

## Introduction

The objective of this project is to analyze a large random sample of Citibike data from the years 2013 to 2019. We will look at different trends between location and month, as well as trends in the type of users and so much more. We will also try to build models that can potentially classify some attribute of a ride. Hopefully after reading this tutorial, you will not only be able to understand more about different statistics/trends about the Citibike data, but also how much Citibike has become integrated in the lives of New Yorkers.

## Required Python Modules
The main python modules we will be using in our project are:
1. folium
2. requests
3. pandas
4. numpy
5. datetime
6. statistics
7. stats_models
8. matplotlib
9. sklearn
10. urllib
11. zipfile
12. warnings

In [ ]:
# !pip install folium

import requests # for get request
import pandas as pd # pandas
import numpy as np # numpy array operations
from datetime import datetime # datetime objects
import matplotlib.pyplot as plt # for plotting
import folium # for adding attributes to maps
import urllib # for retrieving the files from the site
import zipfile # for extracting and compressing the csv files
import statistics # for help with standardizing data
from folium.plugins import HeatMap # for visualizing a heat map for some attributes
import statsmodels.formula.api as smf # for modeling with logistic regression
import statsmodels.api as sm # for modeling with logistic regression
from sklearn.model_selection import train_test_split # for generating training and testing data splits
from sklearn.model_selection import GridSearchCV # for performing k-fold cross validation
from sklearn.neighbors import KNeighborsClassifier # for modeling with k-nearest neighbors

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Data Collection

This is the data collection phase of the data life cycle. During this phase, our focus is on collecting and transforming the data into a usable form. In our case, that usable form is a pandas dataframe derived from a CSV file containing the original data. <br> <br>

After doing a lot of research for a dataset, we started to look towards the travel industry for motivation. We were motivated by data that contained information regarding ride sharing. At the end, we finalized our search on publicly available bike sharing data from Citi Bike NYC. We retrieved the data from the following site: https://s3.amazonaws.com/tripdata/index.html?fbclid=IwAR3BJ9yWYcKtBRYUMQq2SI1IZR9AnFel3C-eTato4mWXtwBz4olhrdrai7Js

The company had available several months worth of data, ranging from June of 2013 to October of 2019, so we decided to explore each of these months. In attempting to utilize the data from the entire data range mentioned, we needed to perform several steps to automate the download and collection process. The following process will describe how we went about collecting and manipulating the data into one aggregate dataframe containing a certain number of entries of each date. <br>

To collect and store the data, we used the following libraries from above:
1. urllib
2. zipfile
3. pandas

As mentioned before, the data was organized by year, and then month, so it was quite predictable in terms of figuring out how to extract the data files from the site. However, there were some unique aspects that required some workarounds. We looped through each year and month pair from June of 2013 to October of 2019. All of the month numbers in the file names had 2 digits, so we need to make sure all months before October (10th month) were **prepended with a "0"**. Most of the zip files followed the format **"{date}{month_2_digit}-citibike-tripdata.zip"**. The remaining zip files had a slightly different name format of **"{date}{month_2_digit}-citibike-tripdata.csv.zip"**. To get work around this, we instituted a check in the loop to determine if the current year was after 2016. If so, then the url and zip file name would be updated accordingly. 

Once those alterations were completed, as necessary, we proceeded to download the file data. To do so, we utilized the library function **urllib.request.urlretrieve** to download the zip file, indicated by the corresponding url, to the corresponding file path location, which was the current directory in this case. With the zip file downloaded, we proceeded to extract the contents of the zip file, only one CSV file in this case, using zipfile **extractall** method and extracted it to the current directory. We also retrieved the name of the file using the zipfile **namelist** and indexing at 0, as there is always only one element (CSV file) in said list. <br>
    
With the CSV data file download and extracted from the corresponding zip file, we transferred it into a pandas dataframe object. We made sure to add year and moth column to differentiate these data points from entries in future CSV files. After that, we decided to add only a sample of the resulting dataframe to our aggregate dataframe. This was due to the fact there were too many entries in every CSV file, so it would be impractical and inefficient to collect every data from each CSV for exploratory data analysis. With that in mind, we decided to randomly sample 10000 rows from each dataframe to be used for data analysis using the pandas **sample** method without replacement. In sampling with n=10000, we get a good representation of the data for that specific date.<br>
    
One issue we ran into when generating each sampled dataframe was with the column (attribute) names. We realized that, after approximately 2017, the attribute names were capitalized. Due to this difference, the resulting aggregate dataframe at first contained many NaNs because pandas concatenates dataframes based on their column names. The dataframe resulted in having 2 times the normal amount of column names: half being lowercase and half being capitalized. Ignoring case, the column names across all CSV files were identical. To work around this, we decided to create a list of default, lowercase column names, **col_names**, and replaced the column names of each sampled dataframe with our list. <br>
    
After iterating over each date, we ended up with a **list of 77 dataframes**, each corresponding to a specific date from the data. Since, we made sure that all of these dataframes had the same column names, we proceeded to concatenate all of them into one aggregate dataframe, as mentioned before. With 77 dataframes, each with 10000 randomly sampled data points, the new aggregate dataframe contained **770001** total data points. To do this, we utilized the pandas **concat** function, because it we could pass an iterable, which, in this case, was our list of sampled dataframes. The resulting dataframe was returned as output for display purposes.

In [ ]:
# The data begins starting on June of 2013. 
year = 2013
month = 6

# In some of the csv files, we found that the column names differed by 
# capitalization, so below is a standard list of the all of the column
# names that we apply to each sampled dataframe. 
col_names = ['tripduration', 'starttime', 'stoptime', 'start station id',
 'start station name', 'start station latitude', 'start station longitude',
 'end station id', 'end station name', 'end station latitude',
 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender', 'year',
 'month']

# A list to hold all of the dataframes
list_dfs = []

# The last csv file contains the data recorded on October of 2019. The
# loop will increment by 1 month, until after this date is reached. 
while (not (year == 2019 and month == 11)):
    date_str = f'{year}{month}'
    
    # If the month is before October (10), prepend the corresponding 
    # integer value with a 0 to make it 2 digits
    if month < 10:
        date_str = f'{year}0{month}'
    
    # Corresponding download URL for each relevant zip file
    url = f'https://s3.amazonaws.com/tripdata/{date_str}-citibike-tripdata.zip'
    # Corresponding name for each relevant zip file
    zip_file = f'{date_str}-citibike-tripdata.zip'
    
    # Format of zip file name changes after 2016, so this if stmt accounts for that
    if (year > 2016):
        url = f'https://s3.amazonaws.com/tripdata/{date_str}-citibike-tripdata.csv.zip'
        zip_file = f'{date_str}-citibike-tripdata.csv.zip'
    
    # Zip file path points to the current directory "."
    zip_file_path = f'./{zip_file}'
    
    print(year, month)
    
    # Download the corresponding zip file
    urllib.request.urlretrieve(url, zip_file)
    
    csv_filename = ''
    
    # Extract the csv from the zip file and get its name
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('.')
        csv_filename = zip_ref.namelist()[0]
    
    # Read the csv file into a pandas dataframe
    df = pd.read_csv(f'./{csv_filename}')
    
    # Add the year and month attributes to distinguish from entries
    # from entries from other dates
    df['year'] = year
    df['month'] = month  
    
    # Take a random sample of 10000 data points from the csv
    sampled_df = df.sample(10000, replace=False, random_state=1)  
    
    # Make sure that all of the dataframes have the same attribute formats
    sampled_df.columns = list(col_names)    
    print(sampled_df.columns.values)
    
    # Add the dataframe to a list containing all dataframes 
    list_dfs.append(sampled_df)   
    
    # Increment by 1 month. Increment the year by 1 if month goes past Dec.
    month += 1  
    if (month > 12):
        month = 1
        year += 1

# Concatenate all dataframes together to form a cumulative dataframe 
bikes_df = pd.concat(list_dfs)     
bikes_df

After creating the dataframe containing sampled observations from every date in the range of June of 2013 to October of 2019, we decided to export it to a CSV file for future use by using the pandas **to_csv** method. The reason behind this was because the process of downloading, extracting, and converting the data from the site was time and memory intensive. Another related thing to note was that kernel did die on occasion and had to restart. For this reason, rather than repeatedly performing these operations every time, we exported to the resulting dataframe to a CSV for efficiency purposes. It will later be converted back to a dataframe object when we begin our data analysis below. 

In [ ]:
# Write the dataframe to a csv file for later use and safe keeping
bikes_df.to_csv('./FINAL_SAMPLED_DATA.csv', index=False)

In [ ]:
# Retrieve the saved dataset (FINAL_SAMPLED_DATA.csv) from zip file here
with zipfile.ZipFile('./FINAL_SAMPLED_DATA.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
    csv_filename = zip_ref.namelist()[0]
bikes_df = pd.read_csv(csv_filename)
bikes_df

Now we can explain what each column in the data set means. It is explained in the list below: 
1. 'tripduration': the bike trip duration measured in seconds
2. 'starttime': the start time of the trip 
3. 'stoptime': the stop time of the trip 
4. 'start station id': the id of the starting Citibike station
5. 'start station name': the name of the starting Citibike station
6. 'start station latitude': the latitude of the starting Citibike station
7. 'start station longitude': the longitude of the starting Citibike station
8. 'end station id': the id of the ending Citibike station
9. 'end station name': the name of the ending Citibike station
10. 'end station latitude': the latitude of the ending Citibike station
11. 'end station longitude': the longitude of the ending Citibike station
12. 'bikeid': the id of the bike
13. 'usertype': customer => 24-hour pass or 3-day pass user; subscriber => Annual Member
14. 'birth year': the birth year of the user 
15. 'gender': 0 => Unknown; 1 => Male; 2 => Female

You can read more about Citibike and the data here:

https://www.citibikenyc.com/system-data?fbclid=IwAR16Bs8wn3wTthMKWMLYDqfoRShF6BzVZMFPCk665ddfJdmgC3hJJ0mP7tw


## Data Processing

For this section, we used the following library from above:
1. pandas

The first thing we wanted to do is remove any incomplete data, especially those that had NaN for locations. For our heatmaps later on, we decided to construct them on the basis of start and end latitudes and longitudes. To do this, we utilizied the pandas **dropna** method to drop all rows that contained NaNs. We felt that this was safer than trying to impute values because, for example, it would be infeasible to replace unknown location values.

In addition, we noticed that several of the years had **"\N"** as an entry, which indicates that the year in the record is unknown. As a result, we resolved to drop those rows from the data, as we were planning on using those values in the data analysis later on. We only included rows that did not have **"\N"** as a value.


In [ ]:
bikes_df = bikes_df.dropna()

bikes_df = bikes_df[bikes_df['birth year'] != '\\N']

bikes_df

We will first turn the starttime and stoptime column values into datetime objects. This will help us later on in the analysis when we need to access individual elements of the time (hour, day, year etc.). The format of the datetime object will be 'year-month-day hour:minute:second:millisecond'. We will be using Python's build-in function 'to_datetime' to help us convert the strings into datetime objects.

In addition, we noticed that some of the data for the years was stored as an object or float, rather than just an int. As a result, we made sure to convert all year values to integers, so that we could use it for later reference. 


In [ ]:
# Convert starttime column values into datetime objects
bikes_df['starttime'] = pd.to_datetime(bikes_df['starttime'])

# Convert stoptime column values into datetime objects
bikes_df['stoptime'] = pd.to_datetime(bikes_df['stoptime'])

# Convert birth year to integer
bikes_df['birth year'] = pd.to_numeric(bikes_df['birth year'], downcast='integer')
bikes_df['year'] = pd.to_numeric(bikes_df['year'], downcast='integer')

# Print the resulting dataframe
bikes_df


## Exploratory Analysis and Data Visualization

For this section, we used the following libraries from above:

1. pandas
2. datetime
3. matplotlib
4. folium
5. numpy

We will be plotting our data and analyzing trends that may be significant. Some plots you will see are most popular starting points of trips, comparison between subscriber and customer of Citibikes, the day that most people use the Citibikes, an analysis on gender of the user, and so much more. 

We will start off by plotting two simple graphs which will help with some analysis later on. The plots will be Frequency of Start Time of Trip and The Most Popular Days of the Week to ride a Citibike.

### Plotting Frequency of Start Time of Trip
We will be plotting the number of occurrences during the day when people begin their trip. 


In [ ]:
time_of_day = {}

# Fill time_of_day hash table with 0 occurrences of users for each hour
for i in range(0, 24):
    time_of_day[i] = 0

# Loop through dataframe and increase counts of users by one 
# for that hour if the user started the trip at that hour
for index, row in bikes_df.iterrows():
    datetime_obj = row['starttime']
    time_of_day[datetime_obj.hour] += 1

# We will now plot histogram of the time of day people start using the bikes.
plt.bar(time_of_day.keys(), time_of_day.values(), align='center')
plt.xlabel('Hour in Day (0-23)')
plt.ylabel('Number of Users Starting Trip')
plt.title('Hour in Day vs. Number of Users Starting Trip')
plt.show()

By looking at the graph above, most of the people rent Citibikes around 8am and 5pm. The reason for these two peaks at these two times could be that these two times correspond to when most people are going to work and leaving work.

### Plotting Most Popular Days of the Week

In [ ]:
week_day= {}

# Fill week_day hash table with 0 occurrences of users for each hour
for i in range(0, 7):
    week_day[i] = 0

# Loop through dataframe and increase counts of users by one 
# for that hour if the user started the trip at that hour
for index, row in bikes_df.iterrows():
    datetime_obj = row['starttime']
    week_day[datetime_obj.weekday()] += 1

# We will now plot histogram of the time of day people start using the bikes.
days = ['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday']
plt.bar(week_day.keys(), week_day.values(), align='center')
plt.xticks(list(week_day.keys()), days, rotation=35)
plt.xlabel('Day of the Week')
plt.ylabel('Number of Users Starting Trip')
plt.title('Day of the Week vs. Number of Users Starting Trip')
plt.show()

### Analyzing Source/Destination Stations
Now, we want to try to find some trend and information about the Citibike stations in NYC. We will first try plotting heatmaps to see if we can find something interesting and then later on, we will plot the top Citibike stations that people go to for their bikes. By plotting these graphs, it will help us learn more about the location of Citibike stations as well as the most popular stations in NYC. 

#### Heatmaps of Source/Destination Stations
First, we will be plotting the heatmap of source and destination locations. By plotting this, we will hopefully be able to analyze stations where most users retrieve and dock their bikes at in NYC. The brighter an area is (more orange/yellow) the more users that were in that area retrieving/docking their bikes. Feel free to zoom in too to take a closer look at the different areas in the city where most people are using Citibikes. 

The following is the Source Stations Heatmap: 

In [ ]:
# The code below will plot the heatmap of source locations 
map_osm = folium.Map(location=[40.7358, -73.9760], zoom_start=11.5)

# Create HeatMap of bike trip starting points in NYC
bikes_df['count'] = 1
HeatMap(data=bikes_df[['start station latitude', 'start station longitude', 'count']]
        .groupby(['start station latitude', 'start station longitude'])
        .sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_osm)

map_osm

The following is the Destination Stations HeatMap: 

In [ ]:
# The code below will plot the heatmap of destination locations 
map_osm = folium.Map(location=[40.7358, -73.9760], zoom_start=11.5)

# Create HeatMap of bike trip ending points in NYC
bikes_df['count'] = 1
HeatMap(data=bikes_df[['end station latitude', 'end station longitude', 'count']]
        .groupby(['end station latitude', 'end station longitude'])
        .sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_osm)

map_osm

By looking at the two maps above, at first glance, we can see that most people start their trips in the heart of Manhattan and Brooklyn. If we zoom in more, we can get a greater sense of where most people start/end their trips from. In the source destination heatmap, we can that around 45th street and the southernest part of Manhattan, there is a good amount of trips that start from the stations near there. Likewise, we can see the same trend in the destination heatmap. Even though the heatmap can tell us which areas in NYC have a lot of people using the Citibikes, we cannot really infer too much about what is happening in those areas. We need to plot the most popular starting/ending stations and see them on the map. By doing this, we can single out the most popular stations in NYC and hopefully understand why these areas on the heatmap are so concentrated.   

#### Plotting Popular Source/Destination Stations
Instead of plotting a heatmap, we can plot the top 10 source stations as well as the top 10 destination stations from all the years 2013-2019. By plotting this, we can see more about where these stations are on the map and see if they correspond to the high concentration of users on the heatmap above. 

In [ ]:
# The two hashes below will store the key as the station name and 
# the value will be the number of occurrences that the key has appeared
# in the dataframe
# stations_start will store the source station counts
# stations_end will store the destination station counts
stations_start = {}
stations_end = {}

# Loop through dataframe and keep count of the number of people who leave 
# from the stations and arrive at the stations
for index, row in bikes_df.iterrows():
    station_name_start = row['start station name']
    station_name_end = row['end station name']
    
    if station_name_start not in stations_start:
        stations_start[station_name_start] = 0
        
    if station_name_end not in stations_end:
        stations_end[station_name_end] = 0
        
    stations_start[station_name_start] += 1
    stations_end[station_name_end] += 1

# Sort the hash by value and retrieve top 10 source/destination stations
# By top 10, we mean the stations that have the most users arriving/departing
# from there
stations_start = sorted(stations_start.items(), key=lambda x:-x[1])[:10]
stations_end = sorted(stations_end.items(), key=lambda x:-x[1])[:10]

# Extract first 10 highest items from each hash
# and fill up the x and y lists that we will plot
top_x_start = []
top_y_start = []
top_x_end = []
top_y_end = []
for key, value in stations_start:
    top_x_start.append(key)
    top_y_start.append(value)
    
for key, value in stations_end:
    top_x_end.append(key)
    top_y_end.append(value)

# We will now plot histogram of the top 10 stations that people start and end at
fig, ax1 = plt.subplots()
ax1.bar(top_x_start, top_y_start, align='center')
ax1.set_xlabel('Station Names')
ax1.set_xticklabels(top_x_start, rotation = 45, ha='right')
ax1.set_ylabel('Number of Users Starting Trip at that Station')
ax1.set_title('Top 10 Source Stations with Most Users')

fig, ax2 = plt.subplots()
ax2.bar(top_x_end, top_y_end, align='center')
ax2.set_xlabel('Station Names')
ax2.set_xticklabels(top_x_end, rotation = 45, ha='right')
ax2.set_ylabel('Number of Users Ending Trip at that Station')
ax2.set_title('Top 10 Destination Stations with Most Users')


From observation, it looks like that the top 10 most popular stations are the same (except in different order) across both graphs. Now since, we know the most popular stations, we will also try to analyze the age of the people who go to these popular stations. This will help us understand more about who uses these bikes in these areas. We will output graphs of age groups in these popular stations. The age groups will be teens: 0-18, young adults: 18-36, middle-aged adults:36-55, and older adults: 55+. The lower bound on the age is inclusive while the upper bound on the age is exclusive for that range.  

In [ ]:
# Add new age column calculated by the current dataset year 
# minus the birth year of the user
bikes_df['age'] = bikes_df['year']-bikes_df['birth year']

# The variables below will correspond to the subplot matrix 
# where i represents the row and j represents the column
i = 0
j = 0

# We will be creating 10 subplots where each plot represents the
# amount of users at that station vs. the age range (defined above) 
fig, ax1 = plt.subplots(5, 2, figsize=(20, 40))

# For each popular station
for name in top_x_start:
    # Retrieve subset of dataframe where the starting station name matches the 
    # current station name we are making plot for    
    current_subset = bikes_df.loc[bikes_df['start station name'] == name]
    
    # Initialize hash where key is the age group and the value is the count for each 
    # age group for the current station we are making plot for     
    age_hash_start = {'teens':0, 'young adults': 0, 'middle-aged adults': 0, 'older adults': 0}
    
    # Load hash
    for index, row in current_subset.iterrows():
        user_age = row['age']
        if user_age < 18:
            age_hash_start['teens'] += 1
        elif user_age < 36:
            age_hash_start['young adults'] += 1
        elif user_age < 55:
            age_hash_start['middle-aged adults'] += 1
        else:
            age_hash_start['older adults'] += 1
            
    # Retrieve subset of dataframe where the ending station name matches the 
    # current station name we are making plot for    
    current_subset = bikes_df.loc[bikes_df['end station name'] == name]
    
    # Reset age group hash because now we will be treating the current station
    # name as the destination station
    age_hash_end = {'teens':0, 'young adults': 0, 'middle-aged adults': 0, 'older adults': 0}
    
    # Load hash
    for index, row in current_subset.iterrows():
        user_age = row['age']
        if user_age < 18:
            age_hash_end['teens'] += 1
        elif user_age < 36:
            age_hash_end['young adults'] += 1
        elif user_age < 55:
            age_hash_end['middle-aged adults'] += 1
        else:
            age_hash_end['older adults'] += 1
    
    # Graph the number of users for each age group for current station. Each age group 
    # will have two bars: one bar will be the number of users treating the current 
    # station as a start station and the other bar will be the number of users treating
    # the current station as an end station
    ind = np.arange(4) 
    ax = ax1[i % 5, j % 2]
    ax.bar(ind, list(age_hash_start.values()), .35, label='Source Station', align='center')
    ax.bar(ind+.35, list(age_hash_end.values()), .35, label='Destination Station', align='center')
    ax.set_xlabel('Age Range')
    ax.set_xticks(ind+.35/2)
    ax.set_xticklabels(list(age_hash_end.keys()))
    ax.set_ylabel('Number of Users')
    ax.set_title('Age range of Users at ' + name + ' Station')
    ax.legend()
    i += 1
    j += 1

# Format plots to make it look nicer
fig.tight_layout()
fig.subplots_adjust(hspace=0.3)
    

By looking at the 10 graphs comparing the number of users to the age group for each popular station, we can see that age groups fluctuate in different stations. For example, in Pershing Square North, we can see that the dominating age group is middle-aged adults while in Lafayette St & E 8 St, the dominating age group is young-adults. Keep this in mind as it will help in our analysis. 

Now we can see what the most popular stations are in the histogram as well as the age groups that appear most at these stations, but now let's see where they are on the map. By doing this, we can look at the stations and try to analyze the environment around that station so that we can understand why that station is so popular. Since the top 10 source and destination stations are the same, we will only provide one map. If you click on a point on the map, it will display the name of the Citibike station. 

In [ ]:
map_osm = folium.Map(location=[40.7300, -74.0007], zoom_start=13)

# Add points to map to show most popular source stations
for name in top_x_start:
    # Retrieve the latitude and longitude of that location in dataframe     
    row = bikes_df.loc[bikes_df['start station name'] == name].iloc[0]
    lat = row['start station latitude']
    long = row['start station longitude']
    
    # Add a circle indicating the location in NYC
    folium.Circle(
        radius=20,
        location=[lat, long],
        popup=name,
        color='black',
        fill=True,
    ).add_to(map_osm)

map_osm


By looking at the graph above, we can see that all of the top 10 stations are located in Manhattan. Also, we can observe that most of these locations are located near very busy bus/metro stations. For example, we can see that the most popular start station (Pershing Square North) is located right next to Grand Central Terminal which is one of the major metro stations in NYC. Likewise, there are Citibike stations right next to Port Authority Bus Terminal and 34th Street Penn Station. These two bus/metro stations are also very busy and popular stations for New Yorkers. Looking at our age range graphs for these stations, we can see that they are mostly middle-aged adults. We could probably infer that most Citibike users in this area use their bike to get to these stations either for work or travel or they leave from these stations after coming back from work. 

You can read more about Grand Central Terminal here:
https://www.grandcentralterminal.com/about/

You can read more about Penn Station here:
https://en.wikipedia.org/wiki/Pennsylvania_Station_(New_York_City)

If we look at the downtown stations, Lafayette St & E 8 St and Broadway & E 14 St, we can see that these Citibike stations are near NYU, a university. By looking at our age range graphs, we can observe that most of the users, are young adults, college students and people who are just starting their careers. If we look at the two stations above those, Broadway & E 22 St and W 21 St & 6 Ave, we can see that those are located near/in the Flatiron District. Baruch College is in the area as well. Therefore, we can see a lot of college students/young adults in this area. We could probably infer that most Citibike users in this area are college students who use the bikes to get to class, dorms, and even restaurants.   

You can read more about Flatiron District here:
https://en.wikipedia.org/wiki/Flatiron_District
 

### Comparing Habits Between Subscribers and Customers (day-pass and single-ride users)

For this section, we used the following libraries from above:
1. pandas
2. datetime
3. matplotlib
4. statistics
5. folium
6. numpy

In [ ]:
customers = bikes_df.loc[bikes_df['usertype'] == "Customer"]
subscribers = bikes_df.loc[bikes_df['usertype'] == "Subscriber"]

#### Customer vs Subscriber Frequency By Day of Week

TODO: Standardize for better comparison

In [ ]:
# compare day of week,
week_day_sub= {}
week_day_cust= {}

# Fill time_of_day hash table with 0 occurrences of users for each hour
for i in range(0, 7):
    week_day_sub[i] = 0
    week_day_cust[i] = 0

# Loop through dataframe and increase counts of users by one 
# for that hour if the user started the trip at that hour
for index, row in bikes_df.iterrows():
    datetime_obj = row['starttime']
    if row['usertype'] == 'Customer':
        week_day_cust[datetime_obj.weekday()] += 1
    elif row['usertype'] == 'Subscriber':
        week_day_sub[datetime_obj.weekday()] += 1

In [ ]:
plt.bar( week_day_sub.keys(), week_day_sub.values(), label = 'Subscribers')
plt.bar( week_day_cust.keys(), week_day_cust.values(), label = 'Non-subscribers')
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))

days = ['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday']
plt.xlabel('Day of the Week')
plt.xticks(list(week_day_cust.keys()), days, rotation=35)
plt.ylabel('Number of Users Starting Trip')
plt.title('Day of the Week vs. Number of Users Starting Trip')
plt.show()

From this graph, we can see several notable trends: 
1. Subscriber usage heavily outweighs non-subscriber usage.
2. Subscriber usage is at its highest during business days (Monday - Friday) and falls off on the weekends.
3. Non-subscriber usage peaks on the weekends (Saturday-Sunday) and falls off during business days. 

To make comparisons between subscribers and non-subscribers more easily, we need to standardize both datasets. To normalize the data, we will be computing the Z score for each the subscriber and non-subscriber datasets using this formula: 

Z = (X - mean) / stddev

In [ ]:
mean_wd_sub = statistics.mean(list(week_day_sub.values()))
stdev_wd_sub = statistics.stdev(list(week_day_sub.values()))
std_wd_sub_vals = [(x - mean_wd_sub)/stdev_wd_sub for x in list(week_day_sub.values())]

mean_wd_cust = statistics.mean(list(week_day_cust.values()))
stdev_wd_cust = statistics.stdev(list(week_day_cust.values()))
std_wd_cust_vals = [(x - mean_wd_cust)/stdev_wd_cust for x in list(week_day_cust.values())]

print(std_wd_sub_vals)
print(std_wd_cust_vals)

In [ ]:
plt.plot( list(week_day_sub.keys()), std_wd_sub_vals, label='Subscribers')
plt.plot( list(week_day_cust.keys()), std_wd_cust_vals, label='Non-subscribers')
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))

days = ['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday']
plt.xlabel('Day of the Week')
plt.xticks(list(week_day_cust.keys()), days, rotation=35)
plt.ylabel('Number of Users Starting Trip (Standardized)')
plt.title('Day of the Week vs. Number of Users Starting Trip Standardized')
plt.show()

This standardized graph shows us that subscriber and non-subscriber usage has an inverse relationship:
Through the business days, subscriber usage increases and non-subscriber usage decreases until wednesday. After Wednesday, subsccriber usage decreases and non-subscriber usage increases through the weekend

#### Customer vs Subscriber Frequency By Time of Day

In [ ]:
# compare time of day

# compare day of week,
time_sub= {}
time_cust= {}

# Fill time_of_day hash table with 0 occurrences of users for each hour
for i in range(0, 24):
    time_sub[i] = 0
    time_cust[i] = 0

# Loop through dataframe and increase counts of users by one 
# for that hour if the user started the trip at that hour
for index, row in bikes_df.iterrows():
    datetime_obj = row['starttime']
    if row['usertype'] == 'Customer':
        time_cust[datetime_obj.hour] += 1
    elif row['usertype'] == 'Subscriber':
        time_sub[datetime_obj.hour] += 1

In [ ]:
plt.bar(list(time_sub.keys()), list(time_sub.values()), label='Subscribers')
plt.bar(list(time_cust.keys()), list(time_cust.values()), label='Non-subscribers')
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))
plt.xlabel('Hour in Day (0-23)')
plt.ylabel('Number of Users Starting Trip')
plt.title('Hour in Day vs. Number of Users Starting Trip')
plt.show()

This graph again shows that:
1. Subscriber usage forms a bimodal distribution, peaking at 8 AM and 5-6 PM.
2. Non-subscriber usage forms a unimodal distribution, peaking between 1-5 PM.

However, the scales for subscribers and non-subscribers are again not the same, so we need to standardize the data as we did before using Z scores.


In [ ]:
mean_hr_sub = statistics.mean(list(time_sub.values()))
stdev_hr_sub = statistics.stdev(list(time_sub.values()))
std_hr_sub_vals = [(x - mean_hr_sub)/stdev_hr_sub for x in list(time_sub.values())]

mean_hr_cust = statistics.mean(list(time_cust.values()))
stdev_hr_cust = statistics.stdev(list(time_cust.values()))
std_hr_cust_vals = [(x - mean_hr_cust)/stdev_hr_cust for x in list(time_cust.values())]

print(std_hr_sub_vals)
print(std_hr_cust_vals)

In [ ]:
plt.plot(list(time_sub.keys()), std_hr_sub_vals, label="Subscribers")
plt.plot(list(time_cust.keys()), std_hr_cust_vals, label="Non-subscribers")
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))
plt.xlabel('Hour in Day (0-23)')
plt.ylabel('Number of Users Starting Trip (Standardized)')
plt.title('Hour in Day vs. Number of Users Starting Trip Standardized')
plt.show()

This graph better shows how usage compares throughout the day between subscribers and non-subcribers. To reiterate the same conclusions as the non-standardized plot:
1. Subscriber usage is low until around 7-9 AM where usage peaks, then decreaes at around 12-3 PM, then peaks again at 5-6 PM before finally falling off again (forming a bimodal distribution).
2. Non-subsriber usage starts low and gradually increases until 2-4 PM where usage peaks, then slowly falls down to lower levels (forming a unimodal distribution.




#### Heatmap of Trips for Customers vs Subscribers

In [ ]:
map_osm = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

bikes_df['count'] = 1
HeatMap(data=customers[['end station latitude', 'end station longitude']]
        .groupby(['end station latitude', 'end station longitude'])
        .sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_osm)

map_osm

In [ ]:
map_osm = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

bikes_df['count'] = 1
HeatMap(data=subscribers[['end station latitude', 'end station longitude']]
        .groupby(['end station latitude', 'end station longitude'])
        .sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_osm)

map_osm

The objective of this heatmap was to see if subscriber and non-subscriber usage was concentrated in specific areas (ie if subscriber usage was centered around office/work districts and if non-subscriber usage was centered around tourist attractions). It appears that end-stop locations between subscribers and non-subsribers is extremely similar. However, if you zoom in closer into the map, you can see that specific stops differ in 'heat'. For example, Penn Station and Grand Central are more popular stops for non-subscribers than it is for subscribers whereas the Spring St.-Lafayette Street stop is more popular for subscribers than it is for non-subscribers.



### Most Popular Stops for Subscribers and Non-subscribers

In [ ]:
sub_pop = {}
sub_pop_cust = {}

cust_pop = {}
cust_pop_sub = {}

# Getting the frequency of each end stop sorted by most frequent
cust_pop = customers['end station name'].value_counts()
sub_pop = subscribers['end station name'].value_counts()

#Standardizing data
mean_sub = statistics.mean(list(sub_pop))
stdev_sub = statistics.stdev(list(sub_pop))
std_sub_vals = [(x,(sub_pop[x] - mean_sub)/stdev_sub) for x in sub_pop.keys()]

mean_cust = statistics.mean(list(cust_pop))
stdev_cust = statistics.stdev(list(cust_pop))
std_cust_vals = [(x, (cust_pop[x] - mean_cust)/stdev_cust) for x in cust_pop.keys()]

# Getting the NON-subscriber frequency of the top 10 subscriber stops
for tup in std_cust_vals[:10]:
    cust_pop_sub[tup[0]] = list(filter(lambda x: tup[0] == x[0], std_sub_vals))[0][1]

# Getting the SUBSCRIBER frequency of the top 10 NON-subscriber stops
for tup in std_sub_vals[:10]:
    sub_pop_cust[tup[0]] = list(filter(lambda x: tup[0] == x[0], std_cust_vals))[0][1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

N = 10
ind = np.arange(N) 

ax.bar(ind, [x[1] for x in std_sub_vals[:10]], width = 0.35, label='subscribers', align='center', color = 'b')
ax.bar(ind + 0.35, list(sub_pop_cust.values()), width = 0.35, label='non-subscribers', align='center', color = 'r')
plt.xticks(ind + 0.35 / 2, list(sub_pop[:10].keys()), rotation = 45, ha='right')
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))
plt.xlabel('Stop Destination Name)')
plt.ylabel('# of Users Ending their Destination at Stop')
plt.title('Most Popular End Stops for Subscribers (compared to non-subcsribers)')

This chart plots the frequency of the top 10 stops for Subscribers as well as the frequencies of the same stops for non-subscribers.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

N = 10
ind = np.arange(N) 

ax.bar(ind, [x[1] for x in std_cust_vals[:10]], width = 0.35, label='non-subscribers', align='center', color='r')
ax.bar(ind + 0.35, list(cust_pop_sub.values()), width = 0.35, label='subscribers', align='center', color='b')
plt.xticks(ind + 0.35 / 2, list(cust_pop[:10].keys()), rotation = 45, ha='right')
plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))
plt.xlabel('Stop Destination Name)')
plt.ylabel('# of Users Ending their Destination at Stop Standardized')
plt.title('Most Popular End Stops for Non-subscribers (compared to subcsribers)', y=1.18)


This chart plots the top 10 most popular stops for non-subscribers and compares it to the normalized popularity of subscribers.

#### Analysis of the two charts:

It appears that the most popular stops for subcribers are not the most popular for non-subcribers. Inversely, the most popular stops for non-subscribers are not the most popular stops for subscribers. 


### Gender Patterns within the Data

The goal of this section is trying to see if we can determine patterns for gender. For example, one of the things we studied was the density of bikeshare start positions and see where each gender is most concentrated. We make conclusions based which areas tend to be the most concentrated by gender. The first thing we did was segregate the data by gender, making on dataframe for males and one for females. One thing to note is that there were 3 gender classifications, with 0 representing unknown. At first, we wanted to include it, as it could be representing genders other than males and females, but we realized that it may also mean that the gender attribute was purely unknown. As a result of this ambiguity, we resolve to exclude unknown from our analysis.

#### Heatmap of Trips for Males and Females

In [ ]:
bikes_df['count'] = 1
males_df = bikes_df[bikes_df['gender'] == 1]
females_df = bikes_df[bikes_df['gender'] == 2]

This code uses the folium **Map** object to initialize a map that is centered at New York City, just south of Manhattan. We then use the **HeatMap** object from the plugins module of folium and use the males dataframe from earlier. We plot the latitude and longitude of each coordinate together, using group_by and sum functions to combine the common observations, and then we add the HeatMap to our existing map and display it. The same was done for the feamle dataframe as well. 

In [ ]:
map_osm_males = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

HeatMap(data=males_df[['start station latitude', 'start station longitude', 'count']]
        .groupby(['start station latitude', 'start station longitude'])
        .sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_osm_males)

map_osm_males

In [ ]:
map_osm_females = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

HeatMap(data=females_df[['start station latitude', 'start station longitude', 'count']]
        .groupby(['start station latitude', 'start station longitude'])
        .sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_osm_females)

map_osm_females

Looking at the maps above, we get some interesting observations at first glance. The maps are pretty similar, but there are some subtle differences. In general, we see that there highly dense (red) areas near Grand Central, 1st Avenue & East 14th Street, Chinatown, etc. This makes sense because Grand Central and Chinatown are especially tourist and travel (work) heavy, so we are bound to see heat map concetrate on those areas. On subtle difference we can see is that near the Brooklyn sports area, specifically the region encompassing Brookyln Nets stadium, it is slightly more concentrated with males than with females. This potentially indicates that males utilize bike share at a higher frequency for the past 6 years than do females when considering this region in Brooklyn. 

Other than this slight difference, which we do not think is very significant, male and female patterns of where they begin their bike share trips are almost identical.

## Modeling: Creating Models to Predict the Gender of a Rider

For this section, we used the following libraries from above:
1. pandas
2. sklearn
3. statsmodels

### Generating Models to Predict Gender

The goal of this section is to figure out whether we can accurate predict the gender of a potential. As a result, we set out to produce a couple of models and compare them as well. 

Creating the heat maps helped us figure out how to determine the our predictors for the logistic regression model. The goal of our model was to see if we could predict the gender, male or female, based on a few factors regarding trip details. Originally, we wanted to include the latitude and longitude in the predictors, but we realized that it was hard to interpret that in the model. Latitude increases going from south to north, while longitude increases going west to east. Despite these facts, we recognized that, as we were centering on a specific region, New York City, there would be very little variation amongst the values. They were very close together, so, for example, a rider starting their trip at a longitude that was 0.01 greater than that of another rider did not seem to be that signifcant in predicting the gender of a rider. While this may not be convincing alone, our heatmaps above defintely confirm this observation. As the heat maps are nearly identical, we see that a difference in latitude and longitude will not be a great determinant in predicting the gender of a potential rider. 

As a result, we decided to create models for predicting gender, based on **starttime, duration, usertype, and age**.

The first thing we will do is prepare our data to be modeled. This involves converting values, so that it can be easy to interpret. 
1. We first converted the **starttime** variable to seconds. To do so we extracted the time portion only from the datetime object and then converted it to seconds. This was because, as the data spanned over several years, the day of which the trip occurred did not matter for the purposes of modeling. We named the variable **startime_sec**. We summed the following:
    - time.hour * 3600 => seconds
    - time.minutes * 60 => seconds
    - time.second <br>
<br>
2. We then calculated the age by using existing attributes. We simply subtracted the **year** of the trip from the **birth year** of the rider of the trip to calculate the age of the rider. We named the variable **age**.
3. We converted the usertype variable to numerical format. While it being a string did not impact the logistic regression modeil, it did prevent the tree from being created.

In [ ]:
# Convert the start times to seconds
bikes_df['starttime_sec'] = bikes_df['starttime'].map(lambda x: (x.hour * 3600) + (x.minute * 60) + x.second)

# Calculate the age by subtracting birth year from year
bikes_df['age'] = bikes_df['year'] - bikes_df['birth year']

# Encode the categorical usertype_code as integers (1 = Subscriber and 0 = Customer)
bikes_df['usertype_code'] = bikes_df['usertype'].map({'Subscriber': 1, 'Customer': 0})

bikes_df

The second thing that we will do is normalize the data set. The reason for this is because we noticed that the **starttime_sec** and **duration** predictors were much larger in magnitude, compared to the rest of the predictors we chose. Another reason for doing so was because we wanted to make sure we could compare their impacts on predicting gender, relatively speaking. It normalizes the distances between the values, putting them on a similar scale.

To do this, we made our own function to standardize a pandas Series object by computing the z-score of each data value. In other words, we took every data item for attribute and computed its z-score. 
<br><br>
**z-score = (value - mean) / (standard deviation)**
<br><br>
We standardized all the predictors this way and created our dataframe.

In [ ]:
# This is a helper function that standardizes an input series object.
# We essentially calculate the z-score for each value in the Series. 
def standardize_attr(series):
    avg = series.mean()
    stdv = series.std()
    series_standardized = (series - avg) / stdv
    return series_standardized

# A list of all the attributes, including the response variable, gender
attrs = ['starttime_sec', 'tripduration', 'age', 'usertype_code', 'gender']

# Extract those relevant attributes from the main bikes_df
norm_bikes_df = bikes_df[attrs]

# Remove all rows where the gender is unknown (0)
norm_bikes_df = norm_bikes_df[norm_bikes_df['gender'] != 0]

# Code the gender such that 0 is Male and Female is 1
norm_bikes_df['gender'] -= 1

# Remove the last 2 attributes before standardizing as they are categorical
# and need no standardization
attrs.pop()
attrs.pop()

# For every continuous attribute, standardize the column of values
for attr in attrs:
    norm_bikes_df[attr] = standardize_attr(norm_bikes_df[attr])
    
norm_bikes_df

The third thing we will do is randomly section out data for training, validation, and test data. We will use a **75-25** split, which corresponds to 75% training data and 25% test data. Before this, we made sure sample out **100000** random observations from the data set. This was because the kernel kept dying when using all over 700000 observations. To do this, we will use the **train_test_split** function from the model_selection module of sklearn. As we are generating a couple of models, we want to be able to compare them on the same dataset, which will be our test data in this case. <br><br>


In [ ]:
# Randomly sample 1000000 attributes from our standardized dataframe
norm_bikes_df = norm_bikes_df.sample(100000)

y = norm_bikes_df['gender']

# Get the training and testing data sets
norm_bikes_train, norm_bikes_test, gender_train, gender_test = train_test_split(norm_bikes_df, y, test_size=0.25)

print(f'The size of our training data is {len(norm_bikes_train)}.')
print(f'The size of our test data is {len(norm_bikes_test)}.')

In [ ]:
norm_bikes_train

In [ ]:
norm_bikes_test

#### Logistic Regression

Now we will generate our model for logistic regression on training data set we created above.

In [ ]:
response = 'gender'
predictors = 'starttime_sec + tripduration + usertype_code + age'

# Make the R-like logistic regression formula for glm
formula = f'{response} ~ {predictors}'

log_reg_model = smf.glm(formula = formula, data=norm_bikes_train, family=sm.families.Binomial())
result = log_reg_model.fit()

# Get the summary statistics
result.summary()

From the logistic regression model we can analyze the coefficients to see determine how they affect the classification of gender. Please note that repeated runs of the training and testing data set splitting, might yield different fits. When we fitted the model repeated times in this fashion, we saw that the majority of models had all significant predictors (near-zero p-values). In the few exceptions, we saw that starttime_sec and, rarely, tripduration were insignificant in explaining the response. We will assume, for the purposes of this analysis, that all the predictors were significant. We performed z-tests (null hypothesis is that the predictor is insignifcant, when accounting for other predictors, and the alternative hypothesis is that the predictor is significant, when accounting for other predictors) on all of them and found that their associated p-values were all less than a significance level of alpha=0.5 (rejected the null hypothesis each time). With the same reasoning, coefficients also change in every rerun of the model. With every model, we saw that intercept, starttime_sec, usertype_code, and age were all negative, while the coefficient of tripduration was positive. We will, as a result, assume that their signs stay the same for the purposes of this analysis. 

Each of the interpretions of the coefficients are as follows:

1. **starttime_sec**: For every additional second later in the day, on average, the odds of a bike rider being classified as female decrease by factor of e^(-0.0214) = 0.979, while holding other predictors fixed. This coefficient is negative, which indicates that it generally decreases the probability of a rider being classified as female. However, the coefficient is near-zero in magnitude, indicating that it has little influence on the probability of a rider being classified as female. 

2. **tripduration**: For every additional second in trip duration, on average, the odds of a bike rider being classified as female increase by factor of e^(0.4521) = 1.572, while holding other predictors fixed. This coefficient is positive, which indicates that it generally increases the probability of a rider being classified as female. The coefficient is larger in magnitude, relative to the other coefficients, indicating that it has a large influence on the probability of a rider being classified as female. 

3. **usertype_code**: Being a Subscriber, on average, the odds of a bike rider being classified as female decrease by factor of e^(-0.4878) = 0.614, when compared to Customers, while holding other predictors fixed. This coefficient is negative, which indicates that it generally decreases the probability of a rider being classified as female. The coefficient is larger in magnitude, relative to the other coefficients, indicating that it has a large influence on the probability of a rider being classified as female. 

4. **age**: For every additional year, on average, the odds of a bike rider being classified as female increase by factor of e^(-0.1217) = 0.885, while holding other predictors fixed. This coefficient is negative, which indicates that it generally decreases the probability of a rider being classified as female. The coefficient is not as large when compared to (2) and (3) in magnitude, relative to the other coefficients, indicating that it has a decent influence on the probability of a rider being classified as female. 


#### Performance of Logistic Regression

To test the accuracy of our model, we evaluated its performance on the test data set we sectioned out earlier. With those probabilites, we then classified each predicted observation, with above 0.5 being classified as female and below being classified as male. We then counted the number of matching observations and then divided that by the total number of observations.


In [ ]:
# Including a list of all the predictors
features = ['starttime_sec', 'tripduration', 'usertype_code', 'age']

# Predict using the test dataset
gender_pred_probs = result.predict(norm_bikes_test[features])

# Each prediction output a probability so finish the prediction
# by classifying each one with a cutoff value of 0.5
gender_pred = [1 if prob > 0.5 else 0 
               for prob in gender_pred_probs]

num_correct = 0
pred_iter = 0

# Find the number of correctly predicted observations
for true_value in gender_test:
    if true_value == gender_pred[pred_iter]:
        num_correct += 1
    pred_iter += 1
    
print(f'There were {num_correct} correctly predicted observations out of {len(gender_pred)}.')
print(f'This yields a model prediction accuracy of about {100 * num_correct / len(gender_pred)}%.')

#### k-Nearest Neighbors (kNN)

Now we will generate our k-nearest neigbors (kNN). We will use libraries to help with corss validation procedure, so that we can generate a tree with optimal number of neighbors. To do this we will utilize sklearn's **DecisionTreeClassifier** object to build the tree and sklearn's **GridSearchCV** object from the model_selection module to perform the 10-fold cross-validation procedure. We will perform this on the training data.

From constructing the kNN model, we first had to tune the hyperparameter, in this case the number of neighbors. To do so, we performed a 10-fold cross validation procedure and tested parameters from 2 to 10, inclusive. We found that the optimal number of neighbors was 8. However, please note that this value may tune differently on repeated runs.


In [ ]:
response = 'gender'

# Including a list of all the predictors
features = ['starttime_sec', 'tripduration', 'usertype_code', 'age']

# We are testing for the best hyperparameter between 2 and 10
param_grid = {'n_neighbors': range(2,10)}

knn = KNeighborsClassifier()

# Perform the 10-fold CV procedure after we fit the model on the next line
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid.fit(X=norm_bikes_train[features], y=gender_train)

# Get the best, most accurate model
best_clf = grid.best_estimator_

# Output the best value of k we selected
grid.best_params_

#### Performance of kNN

To test the accuracy of our model, we evaluated its performance on the test data set we sectioned out earlier. We then counted the number of matching observations and then divided that by the total number of observations.



In [ ]:
# Predict using the test dataset
gender_pred = best_clf.predict(norm_bikes_test[features])

num_correct = 0
pred_iter = 0

# Find the number of correctly predicted observations
for true_value in gender_test:
    if true_value == gender_pred[pred_iter]:
        num_correct += 1
    pred_iter += 1
    
print(f'There were {num_correct} correctly predicted observations out of {len(gender_pred)}.')
print(f'This yields a model prediction accuracy of about {100 * num_correct / len(gender_pred)}%.')

From comparing the predictive capabilities of both of the logistic regression and kNN models, we found that both of them were quite accurate in predicting whether a potenital rider, given their age, duration of trip, start time of trip in the day, and user type. In general, these accuracies are bound to change, but we found that logistic regression did slightly better in its performance. We displayed the accuracy of each model on the test data set in the output of the above cells. 

From the logistic model, we can also determine which predictors are most important in predicting the gender of a rider. From the high coefficients of trip duration and user_type, we would say that these predictors are the most important and influential in predicting the gender of a rider. Being a Subscriber significantly decreases odds, on average, of predicting a rider to be female, when accounting for other predictors. Having a higher trip duration significantly increase odds, on average, of predicting a rider to be female, when accounting for other predictors. 

The nature of the relationship that we are trying to classify is inherently ambiguous, but these models do show that there is some potential in a model being able to classify a rider as male of female correctly. 


## Insight and Conclusions